天池-OGeek算法挑战赛baseline(0.7016)
鱼遇雨欲语与余

In [3]:
# 正文
# 模型使用的lightgbm，共18个特征
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [4]:
train_data = pd.read_table('./data/train.txt', 
        names= ['prefix','query_prediction','title','tag','label'], header= None, encoding='utf-8').astype(str)
val_data = pd.read_table('./data/vali.txt', 
        names = ['prefix','query_prediction','title','tag','label'], header = None, encoding='utf-8').astype(str)
test_data = pd.read_table('./data/test.txt',
        names = ['prefix','query_prediction','title','tag'],header = None, encoding='utf-8').astype(str)

train_data = train_data[train_data['label'] != '音乐' ]
test_data['label'] = -1

train_data = pd.concat([train_data,val_data])

train_data['label'] = train_data['label'].apply(lambda x: int(x))
test_data['label'] = test_data['label'].apply(lambda x: int(x))

D:\Anaconda3\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
items = ['prefix', 'title', 'tag']
for item in items:
    temp = train_data.groupby(item, as_index = False)['label'].agg({item+'_click':'sum', item+'_count':'count'})
    temp[item+'_ctr'] = temp[item+'_click']/(temp[item+'_count'])
    train_data = pd.merge(train_data, temp, on=item, how='left')
    test_data = pd.merge(test_data, temp, on=item, how='left')

In [6]:
for i in range(len(items)):
    for j in range(i+1, len(items)):
        item_g = [items[i], items[j]]
        temp = train_data.groupby(item_g, as_index=False)['label'].agg({'_'.join(item_g)+'_click': 'sum','_'.join(item_g)+'count':'count'})
        temp['_'.join(item_g)+'_ctr'] = temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'count']+3)
        train_data = pd.merge(train_data, temp, on=item_g, how='left')
        test_data = pd.merge(test_data, temp, on=item_g, how='left')
train_data_ = train_data.drop(['prefix', 'query_prediction', 'title', 'tag'], axis = 1)
test_data_ = test_data.drop(['prefix', 'query_prediction', 'title', 'tag'], axis = 1)

In [7]:
print('train beginning')
X = np.array(train_data_.drop(['label'], axis = 1))
y = np.array(train_data_['label'])
X_test_ = np.array(test_data_.drop(['label'], axis = 1))
print('================================')
print(X.shape)
print(y.shape)
print('================================')

train beginning
(2049998, 18)
(2049998,)


In [9]:
xx_logloss = []
xx_submit = []
N = 5
skf = StratifiedKFold(n_splits=N, random_state=42, shuffle=True)
ft=0
fv=0
trainloss=0
valoss=0

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 32,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}

for k, (train_in, test_in) in enumerate(skf.split(X, y)):
    print('train _K_ flod', k)
    X_train, X_test, y_train, y_test = X[train_in], X[test_in], y[train_in], y[test_in]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    gbm = lgb.train(params,lgb_train,num_boost_round=5000,
                    valid_sets=[lgb_train,lgb_eval],early_stopping_rounds=50,verbose_eval=50,)
    trainss=gbm.best_score['training']['binary_logloss']
    valss=gbm.best_score['valid_1']['binary_logloss']
    trainloss+=trainss
    valoss+=valss
    
    ftrain=f1_score(y_train,np.where(gbm.predict(X_train,num_iteration=gbm.best_iteration)>0.5,1,0))
    print(ftrain)
    ft+=ftrain
    
    fval=f1_score(y_test,np.where(gbm.predict(X_test,num_iteration=gbm.best_iteration)>0.5,1,0))
    print(fval)
    fv+=fval
       
    xx_submit.append(gbm.predict(X_test_, num_iteration=gbm.best_iteration))

print('train_logloss_mean:%.5f'%(trainloss/N))
print("val_logloss_mean:%.5f"%(valoss/N))
print('train_f1_mean:%.5f'%(ft/N))
print('val_f1_mean:%.5f'%(fv/N))

train _K_ flod 0
Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.339815	valid_1's binary_logloss: 0.33853
[100]	training's binary_logloss: 0.32465	valid_1's binary_logloss: 0.323392
[150]	training's binary_logloss: 0.32277	valid_1's binary_logloss: 0.32168
[200]	training's binary_logloss: 0.322087	valid_1's binary_logloss: 0.321192
[250]	training's binary_logloss: 0.321728	valid_1's binary_logloss: 0.321023
[300]	training's binary_logloss: 0.321458	valid_1's binary_logloss: 0.320962
Early stopping, best iteration is:
[297]	training's binary_logloss: 0.321469	valid_1's binary_logloss: 0.320959
0.7949417646259457
0.7954178451870855
train _K_ flod 1
Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.339587	valid_1's binary_logloss: 0.338978
[100]	training's binary_logloss: 0.324575	valid_1's binary_logloss: 0.323854
[150]	training's binary_logloss: 0.322649	valid_1's binary_logloss: 0.322045
[200]

In [10]:
s = 0
for i in xx_submit:
    s = s + i

test_data_['label'] = list(s / N)
test_data_['label'] = test_data_['label'].apply(lambda x: round(x))
print(test_data_.label.value_counts())
# print('test_logloss:', np.mean(test_data_.label))
# test_data_['label'].to_csv('./data/Basiline_from_XXY_V1.csv',index = False)

0    34941
1    15059
Name: label, dtype: int64


# 2018/10/20用这个版本递交了一次，10/21 11：00评测得分为0.7021，排名326/2477

# <font color=red> 第一目标：Top10%；第二目标：Top100<font>

# <font color=red>每天11：00更新成绩，可多次递交，但最新版本计算成绩！！！<font>

## 自2018/10/21即使每天递交一个新版本，也就只有17次检验分数的机会！<br><br>所以，建议建立有效的线下评测机制，当均提升时，再提交到线上评测！<br><br>由于10/26-10/31出去旅游，可以把一些没有递交评测的版本留着放到那时候，万一没有东西递交，则递交这些保留的版本看看他们的成绩也可以！

<font color=red>2018/11/1<br>1.使用了lgb.train,而不是lgb.LGBMClassifier.fit()<br>---结论：这二个方法得到的结果应该差不多，顶多是参数设置不同而引起成绩的一些差异<br>2.没有调参<font>